In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/beerdata/df_stratified_sample.csv


In [2]:
df_stratified_s = pd.read_csv("../input/beerdata/df_stratified_sample.csv")
df_stratified_s

,beer_id,beer_name,brewery_id,state_beer,country_beer,style,availability,abv,brewery_name,city,types,look,smell,taste,feel,overall,score
0,1237,Pyramid Hefeweizen,403.0,WA,US,German Hefeweizen,Year round,5.2,"Pyramid Breweries, Inc.",Seattle,Brewery,3.25,3.00,3.00,3.00,3.00,3.02
1,24436,Helio Hefeweizen,7122.0,MT,US,German Hefeweizen,Year round,6.0,Red Lodge Ales,Red Lodge,"Brewery, Bar, Eatery, Beer-to-go",3.75,3.75,3.75,3.75,3.75,3.75
2,4011,Weizen,452.0,NC,US,German Hefeweizen,Year round,4.9,Weeping Radish Bavarian Restaurant,Manteo,"Bar, Eatery",3.50,3.00,4.00,3.50,4.00,3.68
3,2249,Hefeweizen,68.0,MD,US,German Hefeweizen,Year round,4.7,Flying Dog Brewery,Frederick,"Brewery, Bar, Eatery, Beer-to-go",3.50,3.50,3.00,3.50,4.00,3.40
4,4071,Snoweizen,1528.0,CA,US,German Hefeweizen,Year round,5.1,Snowshoe Brewing Company,Arnold,"Brewery, Bar, Eatery",2.50,4.00,3.00,2.50,3.00,3.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,140014,Hipster Brunch Stout,22564.0,MI,US,Stout,Fall,11.0,Odd Side Ales,Grand Haven,"Brewery, Bar, Eatery, Beer-to-go",4.50,4.50,4.25,4.00,4.25,4.30
39996,54320,Steam Fired Stout,21554.0,OR,US,Stout,Year round,7.6,Fire Mountain Brew House,Carlton,"Brewery, Beer-to-go",4.00,4.00,4.00,4.00,4.00,4.00
39997,1879,Samuel Adams Cream Stout,35.0,MA,US,Stout,Year round,4.9,Boston Beer Company (Samuel Adams),Jamaica Plain,"Brewery, Beer-to-go",4.00,4.00,4.00,4.00,4.00,4.00
39998,101578,Coffee Abraxas,25888.0,MO,US,Stout,Rotating,10.0,Perennial Artisan Ales,Saint Louis,"Brewery, Bar, Eatery, Beer-to-go",4.75,4.50,4.50,4.25,4.50,4.49


In [3]:
#Encode string values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
encoded_df_sample = df_stratified_s.copy()
encoded_df_sample["state_beer"] = le.fit_transform(encoded_df_sample["state_beer"])
encoded_df_sample["availability"] = le.fit_transform(encoded_df_sample["availability"])
encoded_df_sample["types"] = le.fit_transform(encoded_df_sample["types"])
encoded_df_sample["style"] = le.fit_transform(encoded_df_sample["style"])
encoded_df_sample = encoded_df_sample.drop(['beer_name', 'country_beer','brewery_name','city','brewery_id','beer_id','smell','look','feel','score','overall'], axis=1)
encoded_df_sample

,state_beer,style,availability,abv,types,taste
0,47,18,6,5.2,1,3.00
1,26,18,6,6.0,5,3.75
2,27,18,6,4.9,0,4.00
3,20,18,6,4.7,5,3.00
4,4,18,6,5.1,4,3.00
...,...,...,...,...,...,...
39995,22,36,0,11.0,5,4.25
39996,37,36,6,7.6,10,4.00
39997,19,36,6,4.9,10,4.00
39998,24,36,2,10.0,5,4.50


In [4]:
# import dependences
import numpy as np
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [5]:
#Data and target 
y = encoded_df_sample['style'].values
X = encoded_df_sample.copy().drop(columns =['style'])

In [6]:
#Scale the input data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
import optuna
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [9]:
# hyperparameter tuning using optuna
# reference: https://www.kaggle.com/code/hamzaghanmi/xgboost-catboost-using-optuna/notebook?scriptVersionId=94510532
def objective(trial,data=X_scaled,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.33,random_state=1)
    
    param = {
        'tree_method':'gpu_hist', 
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 200,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    accuracy = sklearn.metrics.accuracy_score(test_y, preds)
    
    return accuracy

In [10]:
# run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-06-05 13:58:49,673] A new study created in memory with name: no-name-25e955e1-5c52-47c7-9221-8db725f5de7f
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 13:59:07,478] Trial 0 finished with value: 0.14068181818181819 and parameters: {'lambda': 1.2212397088555664, 'alpha': 0.327260173408721, 'colsample_bytree': 0.8, 'subsample': 0.8, 'learning_rate': 0.01, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 177}. Best is trial 0 with value: 0.14068181818181819.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 13:59:12,053] Trial 1 finish

Number of finished trials: 50
Best trial: {'lambda': 0.02128232993738036, 'alpha': 0.0010978213771125056, 'colsample_bytree': 0.9, 'subsample': 0.6, 'learning_rate': 0.018, 'max_depth': 13, 'random_state': 2020, 'min_child_weight': 1}


In [11]:
#record trail history
hist = study.trials_dataframe()
hist.head()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.140682,2022-06-05 13:58:49.675403,2022-06-05 13:59:07.477906,0 days 00:00:17.802503,0.327260,0.8,1.221240,0.010,17,177,2020,0.8,COMPLETE
1,1,0.157955,2022-06-05 13:59:07.481390,2022-06-05 13:59:12.053186,0 days 00:00:04.571796,3.087065,0.3,0.003549,0.012,5,132,2020,0.7,COMPLETE
2,2,0.252197,2022-06-05 13:59:12.055169,2022-06-05 13:59:21.359961,0 days 00:00:09.304792,1.289423,0.9,0.199426,0.020,13,81,2020,0.7,COMPLETE
3,3,0.235530,2022-06-05 13:59:21.362230,2022-06-05 13:59:29.823749,0 days 00:00:08.461519,0.176333,0.4,0.010107,0.008,9,57,2020,0.7,COMPLETE
4,4,0.090758,2022-06-05 13:59:29.827542,2022-06-05 13:59:33.672704,0 days 00:00:03.845162,0.163113,0.7,1.012868,0.016,5,228,2020,0.5,COMPLETE


Attempt 2

In [14]:
# hyperparameter tuning using optuna
# reference: https://www.kaggle.com/code/hamzaghanmi/xgboost-catboost-using-optuna/notebook?scriptVersionId=94510532
def objective(trial,data=X_scaled,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.33,random_state=1)
    
    param = {
        'tree_method':'gpu_hist', 
        'lambda': trial.suggest_float("lambda", 1e-5, 10.0, log=True),
        'alpha': trial.suggest_float("alpha", 1e-5, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': trial.suggest_int('n_estimators',0, 1000,step = 100),
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
        'random_state': trial.suggest_categorical('random_state', [2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    accuracy = sklearn.metrics.accuracy_score(test_y, preds)
    
    return accuracy

In [15]:
# run the study
study2 = optuna.create_study(direction='maximize')
study2.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study2.trials))
print('Best trial:', study2.best_trial.params)
print('Best value:', study2.best_trial.value)

[I 2022-06-05 14:31:54,246] A new study created in memory with name: no-name-1c7f5e74-a795-4963-b2a0-29e31189c822
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 14:32:13,294] Trial 0 finished with value: 0.1762878787878788 and parameters: {'lambda': 0.0004071510472554416, 'alpha': 0.007704226736559003, 'colsample_bytree': 0.7, 'subsample': 0.6, 'learning_rate': 0.016, 'n_estimators': 900, 'max_depth': 9, 'random_state': 2020, 'min_child_weight': 161}. Best is trial 0 with value: 0.1762878787878788.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 14:

Number of finished trials: 50
Best trial: {'lambda': 0.35573812114503994, 'alpha': 0.0002588872721826578, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.012, 'n_estimators': 900, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 1}
Best value: 0.6446212121212122


In [16]:
print('Best trial:', study2.best_trial.params)
#record trail history
hist = study2.trials_dataframe()
hist.head()

Best trial: {'lambda': 0.35573812114503994, 'alpha': 0.0002588872721826578, 'colsample_bytree': 0.8, 'subsample': 0.5, 'learning_rate': 0.012, 'n_estimators': 900, 'max_depth': 17, 'random_state': 2020, 'min_child_weight': 1}


,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.140682,2022-06-05 13:58:49.675403,2022-06-05 13:59:07.477906,0 days 00:00:17.802503,0.327260,0.8,1.221240,0.010,17,177,2020,0.8,COMPLETE
1,1,0.157955,2022-06-05 13:59:07.481390,2022-06-05 13:59:12.053186,0 days 00:00:04.571796,3.087065,0.3,0.003549,0.012,5,132,2020,0.7,COMPLETE
2,2,0.252197,2022-06-05 13:59:12.055169,2022-06-05 13:59:21.359961,0 days 00:00:09.304792,1.289423,0.9,0.199426,0.020,13,81,2020,0.7,COMPLETE
3,3,0.235530,2022-06-05 13:59:21.362230,2022-06-05 13:59:29.823749,0 days 00:00:08.461519,0.176333,0.4,0.010107,0.008,9,57,2020,0.7,COMPLETE
4,4,0.090758,2022-06-05 13:59:29.827542,2022-06-05 13:59:33.672704,0 days 00:00:03.845162,0.163113,0.7,1.012868,0.016,5,228,2020,0.5,COMPLETE


In [18]:
optuna.visualization.plot_optimization_history(study2)

Trial 3

In [10]:
# hyperparameter tuning using optuna
# reference: https://www.kaggle.com/code/hamzaghanmi/xgboost-catboost-using-optuna/notebook?scriptVersionId=94510532
def objective(trial,data=X_scaled,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.33,random_state=1)
    
    param = {
        'tree_method':'gpu_hist', 
        'lambda': trial.suggest_float("lambda", 1e-5, 10.0, log=True),
        'alpha': trial.suggest_float("alpha", 1e-5, 10.0, log=True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 200,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17]),
            }
    model = xgb.XGBClassifier(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    accuracy = sklearn.metrics.accuracy_score(test_y, preds)
    
    return accuracy

In [11]:
# run the study
study3 = optuna.create_study(direction='maximize')
study3.optimize(objective, n_trials=40)
print('Number of finished trials:', len(study3.trials))
print('Best trial:', study3.best_trial.params)
print('Best value:', study3.best_trial.value)

[I 2022-06-05 19:45:19,495] A new study created in memory with name: no-name-aa3712b1-5fb3-49e7-85a3-ed4e0eea3628
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 19:45:46,515] Trial 0 finished with value: 0.42098484848484846 and parameters: {'lambda': 6.303103387005962e-05, 'alpha': 4.326504599700511, 'colsample_bytree': 0.5, 'subsample': 0.5, 'learning_rate': 0.018, 'max_depth': 17}. Best is trial 0 with value: 0.42098484848484846.
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,
[I 2022-06-05 19:46:54,097] Trial 1 finished with value: 0.5328030303030303 and param

Number of finished trials: 40
Best trial: {'lambda': 0.015488424655677365, 'alpha': 5.91200269888891e-05, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 13}
Best value: 0.6439393939393939


In [12]:
#record trail history
print('Best trial:', study3.best_trial.params)
hist = study3.trials_dataframe()
hist.head()

Best trial: {'lambda': 0.015488424655677365, 'alpha': 5.91200269888891e-05, 'colsample_bytree': 0.9, 'subsample': 1.0, 'learning_rate': 0.02, 'max_depth': 13}


,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_subsample,state
0,0,0.420985,2022-06-05 19:45:19.499553,2022-06-05 19:45:46.514813,0 days 00:00:27.015260,4.326505,0.5,0.000063,0.018,17,0.5,COMPLETE
1,1,0.532803,2022-06-05 19:45:46.516795,2022-06-05 19:46:54.096820,0 days 00:01:07.580025,0.083994,0.4,0.000221,0.018,17,0.5,COMPLETE
2,2,0.381439,2022-06-05 19:46:54.098666,2022-06-05 19:47:07.541923,0 days 00:00:13.443257,0.000017,0.5,0.246687,0.008,5,0.8,COMPLETE
3,3,0.522197,2022-06-05 19:47:07.543953,2022-06-05 19:47:33.888255,0 days 00:00:26.344302,0.989513,1.0,0.000191,0.014,7,0.6,COMPLETE
4,4,0.454318,2022-06-05 19:47:33.892285,2022-06-05 19:48:01.402033,0 days 00:00:27.509748,0.039716,0.7,4.285139,0.008,7,0.8,COMPLETE


In [13]:
# Plot optimization histroy of all trials as well as the best score
optuna.visualization.plot_optimization_history(study3)

In [14]:
#plot the parameter relationshiip as slice
optuna.visualization.plot_slice(study3)

In [15]:
#Visualize parameter importances
optuna.visualization.plot_param_importances(study3)